In [1]:
from enum import Enum
import random 
import matplotlib.pylab as plt
import scipy.sparse as sparse
import numpy as np
import csv
from celluloid import Camera

max_x = 100
max_y = 100
Matrix = [[None for x in range(max_x)] for y in range(max_y)] 
pop_size = 1000
init_infect_rate = 0.002
death_rate = 0.03
mean_infect_duration = 10
mobilitiy = 100 # %
stat_rate = -0.01  # population stationary rate 0-1
num_of_runs = 10000
# time_to_reach_max_infected_rate = 0
# death_count = 0
# infected_count = 0
# max_infected_rate = 0.0


class Status(Enum):
    HEALTH = 1
    INFECTED = 2
    IMMUNE = 3
    DIED = 4
    


# import sys  
# !{sys.executable} -m pip install scipy

class Location:
    def __init__(self):
        while True:
            self.x = random.randrange(0,100)
            self.y = random.randrange(0,100) # generate number from 0 to 99
            if Matrix[self.x][self.y] is None:
                break
    def set_location(self, x, y):
        self.x = x
        self.y = y

class Direction:
    def __init__(self):
        self.x = random.choice([-1,0,1])
        self.y = random.choice([-1,0,1])
        while self.x == 0 and self.y == 0:
            self.x = random.choice([-1,0,1])
            self.y = random.choice([-1,0,1])
    
    def reset(self):
        self.x = random.choice([-1,0,1])
        self.y = random.choice([-1,0,1])
        while self.x == 0 and self.y == 0:
            self.x = random.choice([-1,0,1])
            self.y = random.choice([-1,0,1])
        

class Person:
    def __init__(self):
        self.location = Location()
        self.can_move = True
        self.status = Status.HEALTH
        self.move_dir = Direction()
        self.num_days_after_infect = 0
        self.infect_duration = np.random.exponential(mean_infect_duration) 
    
    # return False if died
    # return True if alive
    def check_status(self):
        global death_count
        if self.status == Status.INFECTED:
            self.num_days_after_infect += 1
            # TODO: need to transfer mean infect duration to specific infect duration
            if self.num_days_after_infect >= self.infect_duration:
                # DIED or IMMUNE
                if (random.randrange(0,100)) < (death_rate*100):
                    self.status = Status.DIED
               #     death_count += 1
                    return False
                else:
#                     # what if 90% people get immune
#                     if(random.randrange(0,100) < 50):
                    self.status = Status.IMMUNE
#                     else:
#                         self.status = Status.HEALTH
                    self.num_days_after_infect = 0
                    return True
        return True
        
    def move(self):
        global infected_count
        if not self.can_move or random.randrange(0,100) > mobilitiy:
            return
        
        new_y = self.location.y + self.move_dir.y
        new_x = self.location.x + self.move_dir.x

        # check if out of bound
        if new_y >= max_y or new_y < 0 or new_x >= max_x or new_x < 0:
            # change direction
            self.move_dir.reset()

        # check if collision with others
        elif not Matrix[new_x][new_y] is None:
            someone = Matrix[new_x][new_y]
            # if self is health
            if self.status == Status.HEALTH:
                # if the other one is infected, then self will be infected by someone
                if someone.status == Status.INFECTED:
                    self.status = Status.INFECTED
                #    infected_count += 1
            # if self is infected
            elif self.status == Status.INFECTED:
                # if some one is infected, then he will be infected by self
                if someone.status == Status.HEALTH:
                    someone.status = Status.INFECTED
                  #  infected_count += 1
                    
            # change direction
            self.move_dir.reset()
            
        else: # no collision
            Matrix[new_x][new_y] = Matrix[self.location.x][self.location.y]
            Matrix[self.location.x][self.location.y] = None
            self.location.set_location(new_x, new_y)
            
            # correctness: if self is infected, people in front of him will be infected too
            if self.status == Status.INFECTED:
                new_y = self.location.y + self.move_dir.y
                new_x = self.location.x + self.move_dir.x
                if not(new_y >= max_y or new_y < 0 or new_x >= max_x or new_x < 0) and not Matrix[new_x][new_y] is None:
                    someone = Matrix[new_x][new_y]
                    if someone.status == Status.HEALTH:
                        someone.status = Status.INFECTED
            
                        

def draw_graph(turn):
   # print("turn:",turn)
    # create a figure and axis
    #if turn%50 == 0:
#     fig, ax = plt.subplots()
#     plt.figure(figsize=(8, 6), dpi=80)
    # plot each data-point
    count = 0
    health_count = 0
    infect_count = 0
    immune_count = 0
    death_count = 0
    colors = {'HEALTH':'g', 'INFECTED':'r', 'IMMUNE':'b'}
    for i in range(max_x):
        for j in range(max_y):
            if not Matrix[i][j] is None:
                person = Matrix[i][j]
                if (person.status == Status.INFECTED):
                    infect_count += 1
                elif person.status == Status.HEALTH:
                    health_count += 1
                elif person.status == Status.DIED:
                    death_count += 1
                elif person.status == Status.IMMUNE:
                    immune_count += 1
                #if turn%50 == 0:
                
               # plt.scatter(person.location.x,person.location.y,s=10,color=colors[person.status.name])
#                camera.snap()
                count += 1
#     title = 'period ' +str(turn)
#     plt.title(title)
#     plt.savefig('s0_1/s0_'+str(turn)+'.png')
    
    return infect_count, pop_size - infect_count - health_count - immune_count, health_count
    
#     print("Total infected people count = ", infect_count )
#     print("Total health people count = ", health_count )
#     print("Total immune people count = ", immune_count )
#     print("Total died people count = ", death_count )
#     print("Total people count = ",count)
    
    # set a title and labels
#     if turn%50 == 0:
   
   
def main():
    # global max_infected_rate
    people = []
    global Matrix
    Matrix = [[None for x in range(max_x)] for y in range(max_y)] 
    for _ in range(pop_size):
        person = Person()
        Matrix[person.location.x][person.location.y] = person
        people.append(person)
    
    init_infected_count = int(init_infect_rate * pop_size)
   # print(len(people))
   # print('total people = ', pop_size)
   # print('init infect count =',init_infected_count)
    # since location of a person is randomly assigned
    # we can choose the first k person as infected person to remain randomness
    infect_indexes = random.sample(range(pop_size), init_infected_count)
    for i in infect_indexes:
        people[i].status = Status.INFECTED
    
    state_count = int(stat_rate * pop_size)
  #  print('total number of state people = ', state_count)
    state_indexes = random.sample(range(pop_size), state_count)
    for i in state_indexes:
        people[i].can_move = False;
    
    total_infected_count = 0
    infected_count = 0
    max_infected_rate = 0.0 
    time_to_reach_max_infected_rate = 0
    death_count = 0
    
    draw_graph(-1)
   # camera = Camera(plt.figure())
    for run in range(num_of_runs):
        for i in range(pop_size):
            someone = people[i]
            
            if someone.status == Status.DIED: # someone already died, skip
              #  print (i,' has died')
                continue
            if not someone.check_status(): # someone died in this turn, remove it from map
              #  print (i,' died')
                Matrix[someone.location.x][someone.location.y] = None
            someone.move()
        
        infected_count, death_count, health_count = draw_graph(run)
        
        if (infected_count * 1.0 / (pop_size - death_count ) > max_infected_rate):
            max_infected_rate = infected_count * 1.0 / (pop_size - death_count)
            time_to_reach_max_infected_rate = run
        total_infected_count = pop_size - health_count
        
        if (infected_count == 0):
            return  max_infected_rate, time_to_reach_max_infected_rate, total_infected_count, death_count, run
            

    print('max_infected_rate',max_infected_rate)
    print('time_to_reach_max_infected_rate',time_to_reach_max_infected_rate)
    print('infected_count',infected_count)
    print('total_infected_count',total_infected_count)
    print('death_count',death_count)
   
                
        
     
    

if __name__ == "__main__":
    results = []
    for _ in range(101):
        
        result = {}
        stat_rate += 0.01
        print('s = {}'.format(stat_rate))
        result['stat_rate'] = stat_rate
        count = 100000
        sum_max_infected_rate, sum_time_to_reach_max_infected_rate, sum_total_infected_count, sum_death_count, sum_stop_run = 0.0,0.0,0.0,0.0,0.0
        for i in range(count):
    #         global infected_count
    #         global death_count
    #         global stat_rate
     #       stat_rate = 0
    #         print('state_rate = ', 0)
    #         infected_count = 0
    #         death_count = 0
      #      print('test {}'.format(i))
            max_infected_rate, time_to_reach_max_infected_rate, total_infected_count, death_count, stop_run = main()
            sum_max_infected_rate += max_infected_rate
            sum_time_to_reach_max_infected_rate += time_to_reach_max_infected_rate
            sum_total_infected_count += total_infected_count
            sum_death_count += death_count
            sum_stop_run += stop_run
        result['ave_infected_rate'] = sum_total_infected_count/count/pop_size
        result['ave_death_rate'] = sum_death_count/count/pop_size
        result['ave_stop_run'] = sum_stop_run/count
        result['ave_max_infected_rate'] = sum_max_infected_rate/count
        result['ave_time_to_reach_max_infected_rate'] = sum_time_to_reach_max_infected_rate/count
        results.append(result)
#         print('ave_infected_rate',sum_total_infected_count/count/pop_size)
#         print('ave_death_count',sum_death_count/count)
#         print('ave_stop_run',sum_stop_run/count)
#         print('ave_max_infected_rate',sum_max_infected_rate/count)
#         print('ave_time_to_reach_max_infected_rate',sum_time_to_reach_max_infected_rate/count)
        
    with open('names_001200.csv', 'w') as csvfile:
        fieldnames = ['s', 'ave_infected_rate', 'ave_death_rate', 'ave_stop_run', 'ave_max_infected_rate', 'ave_time_to_reach_max_infected_rate']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for result in results:
            writer.writerow({'s': str(result['stat_rate']), 
                             'ave_infected_rate': str(result['ave_infected_rate']), 
                             'ave_death_rate': str(result['ave_death_rate']),
                             'ave_stop_run': str(result['ave_stop_run']),
                             'ave_max_infected_rate': str(result['ave_max_infected_rate']),
                             'ave_time_to_reach_max_infected_rate': str(result['ave_time_to_reach_max_infected_rate'])})
        
    
    
    

s = 0.0
s = 0.01
s = 0.02
s = 0.03
s = 0.04
s = 0.05
s = 0.060000000000000005
s = 0.07
s = 0.08
s = 0.09
s = 0.09999999999999999
s = 0.10999999999999999
s = 0.11999999999999998
s = 0.12999999999999998
s = 0.13999999999999999
s = 0.15
s = 0.16
s = 0.17
s = 0.18000000000000002
s = 0.19000000000000003
s = 0.20000000000000004
s = 0.21000000000000005
s = 0.22000000000000006
s = 0.23000000000000007
s = 0.24000000000000007
s = 0.25000000000000006
s = 0.26000000000000006
s = 0.2700000000000001
s = 0.2800000000000001
s = 0.2900000000000001
s = 0.3000000000000001
s = 0.3100000000000001
s = 0.3200000000000001
s = 0.3300000000000001
s = 0.34000000000000014
s = 0.35000000000000014
s = 0.36000000000000015
s = 0.37000000000000016
s = 0.38000000000000017
s = 0.3900000000000002
s = 0.4000000000000002
s = 0.4100000000000002
s = 0.4200000000000002
s = 0.4300000000000002
s = 0.4400000000000002
s = 0.45000000000000023
s = 0.46000000000000024
s = 0.47000000000000025
s = 0.48000000000000026
s = 0.49000000000